# HummingBird Demo

In [54]:
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from hummingbird.ml import convert

# Create some random data for binary classification
from sklearn import datasets


X, y = datasets.make_classification(n_samples=100000, n_features=28)


In [55]:
# Create and train a model (scikit-learn RandomForestClassifier in this case)
skl_model = RandomForestClassifier(n_estimators=1000, max_depth=10)
skl_model.fit(X, y)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=10, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=1000,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [56]:
# Using Hummingbird to convert the model to PyTorch
model = convert(skl_model, 'pytorch')

In [62]:
type(model)

hummingbird.ml._container.PyTorchBackendModel

In [58]:
model.to('cuda')

PyTorchBackendModel(
  (operator_map): ModuleDict(
    (SklearnRandomForestClassifier): PerfectTreeTraversalDecisionTreeImpl(
      (nodes): ParameterList(
          (0): Parameter containing: [torch.cuda.LongTensor of size 2000 (GPU 0)]
          (1): Parameter containing: [torch.cuda.LongTensor of size 4000 (GPU 0)]
          (2): Parameter containing: [torch.cuda.LongTensor of size 8000 (GPU 0)]
          (3): Parameter containing: [torch.cuda.LongTensor of size 16000 (GPU 0)]
          (4): Parameter containing: [torch.cuda.LongTensor of size 32000 (GPU 0)]
          (5): Parameter containing: [torch.cuda.LongTensor of size 64000 (GPU 0)]
          (6): Parameter containing: [torch.cuda.LongTensor of size 128000 (GPU 0)]
          (7): Parameter containing: [torch.cuda.LongTensor of size 256000 (GPU 0)]
          (8): Parameter containing: [torch.cuda.LongTensor of size 512000 (GPU 0)]
      )
      (biases): ParameterList(
          (0): Parameter containing: [torch.cuda.FloatTens

# Timing Various Runs

## Batch

In [57]:
%%timeit
skl_model.predict_proba(X)

9.58 s ± 40.2 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [59]:
%%timeit
model.predict_proba(X)

195 ms ± 436 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [68]:
speedup = 9580/195
print(f'A speedup of {np.round(speedup)}x')

A speedup of 49.0x


## Single

In [67]:
%%timeit
skl_model.predict_proba(np.array([X[0]]))

79.6 ms ± 1.28 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [66]:
%%timeit
model.predict_proba(np.array([X[0]]))

1.28 ms ± 12.9 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [69]:
speedup = 79.6/1.6
print(f'A speedup of {np.round(speedup)}x')

A speedup of 50.0x


# Comparing Preds

In [70]:
scikit_preds = skl_model.predict_proba(X)

In [71]:
hb_preds = model.predict_proba(X)

In [72]:
scikit_preds

array([[0.97264287, 0.02735713],
       [0.98037972, 0.01962028],
       [0.77330629, 0.22669371],
       ...,
       [0.97787525, 0.02212475],
       [0.75329705, 0.24670295],
       [0.0160022 , 0.9839978 ]])

In [73]:
hb_preds

array([[0.97264284, 0.02735713],
       [0.98037976, 0.01962028],
       [0.7733063 , 0.22669372],
       ...,
       [0.9778753 , 0.02212475],
       [0.75329703, 0.24670295],
       [0.01600221, 0.9839978 ]], dtype=float32)

In [86]:
scikit_1_0 = scikit_preds[:,1]>0.5 
hb_1_0 = hb_preds[:,1]>0.5 
len(scikit_1_0) == sum(scikit_1_0==hb_1_0)

True